# json 파일 병합

In [1]:
import json
import os
import glob

# JSON 파일 병합 함수
def merge_json_files(input_dir, output_file):
    merged_data = []
    json_pattern = os.path.join(input_dir, '*.json')
    file_list = glob.glob(json_pattern)
    
    for file in file_list:
        with open(file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            merged_data.append(data)
    
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(merged_data, f, indent=4)

# 입력 디렉토리와 출력 파일 경로 설정
input_directory = 'path/to/your/json/files'
output_file = 'merged_labels.json'

# JSON 파일 병합 실행
merge_json_files(input_directory, output_file)
print(f"Merged JSON data written to '{output_file}'")


Merged JSON data written to 'merged_labels.json'


# copilot
OCR json 파일 요소 중 bounding box 값이 단어 별로 구분되어 있는 경우, python에서 crnn 모델을 이용해 text 구분을 하는 코드를 짜줄 수 있어?

In [ ]:
# 필요한 라이브러리 설치
# !pip install tensorflow opencv-python numpy matplotlib

# 라이브러리 임포트
import tensorflow as tf
import numpy as np
import cv2
import json
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K


In [ ]:
# JSON 파일 로드 함수
def load_json(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

# 이미지 전처리 함수
def preprocess_image(image_path, bounding_box):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    x, y, w, h = bounding_box
    cropped_image = image[y:y+h, x:x+w]
    resized_image = cv2.resize(cropped_image, (128, 32))  # CRNN 입력 크기에 맞게 조정
    normalized_image = resized_image / 255.0
    return normalized_image

# JSON 파일 경로
json_path = 'path/to/your/label.json'
data = load_json(json_path)

# 이미지 경로
image_path = data['file_name']

# 각 단어의 바운딩 박스와 텍스트 정보
words_info = data['words']

# 전처리된 이미지 리스트
processed_images = []
for word_info in words_info:
    bounding_box = word_info['bounding_box']
    processed_image = preprocess_image(image_path, bounding_box)
    processed_images.append(processed_image)


In [ ]:
# CTC 디코딩 함수
def ctc_decode(predictions, charset):
    decoded = K.ctc_decode(predictions, input_length=np.ones(predictions.shape[0]) * predictions.shape[1])[0][0]
    decoded_text = K.get_value(decoded)
    return ''.join([charset[i] for i in decoded_text if i != -1])

# CRNN 모델 정의 (간단한 예제)
def build_crnn_model(input_shape, num_classes):
    input_layer = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Reshape((-1, x.shape[-1]))(x)
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(x)
    x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.models.Model(inputs=input_layer, outputs=x)
    return model

# 모델 입력 크기와 클래스 수 정의
input_shape = (32, 128, 1)
num_classes = 37  # 예: 알파벳 26자 + 숫자 10자 + 공백 1자

# 모델 빌드
model = build_crnn_model(input_shape, num_classes)

# 모델 요약 출력
model.summary()

# 예측 수행 및 CTC 디코딩
charset = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 '  # 예시 문자셋
for processed_image in processed_images:
    processed_image = np.expand_dims(processed_image, axis=-1)  # 채널 차원 추가
    processed_image = np.expand_dims(processed_image, axis=0)  # 배치 차원 추가
    predictions = model.predict(processed_image)
    
    # CTC 디코딩을 통해 실제 텍스트 판독
    predicted_text = ctc_decode(predictions, charset)
    print(f"Predicted text: {predicted_text}")


## 한글 문자셋 만드는 부분

In [ ]:
# 한글 음절 셋 생성 코드
def generate_hangul_syllables():
    hangul_syllables = []
    for cho in range(0x1100, 0x1113):  # 초성 (19자)
        for jung in range(0x1161, 0x1176):  # 중성 (21자)
            for jong in range(0x11A8, 0x11C3):  # 종성 (27자)
                syllable = chr(0xAC00 + (cho - 0x1100) * 21 * 28 + (jung - 0x1161) * 28 + (jong - 0x11A7))
                hangul_syllables.append(syllable)
    return hangul_syllables

# 한글 음절 셋 생성
hangul_syllables = generate_hangul_syllables()

# 생성된 한글 음절 셋 출력 (일부 예시)
print(hangul_syllables[:100])  # 처음 100개의 음절 출력
